
# **Lab 1: Recommendation System and Matrix Factorization**
 
*Part 1:* Topic extraction with NMF

 *Part 2:* Gradient Descent of MF

##  Topic extraction with NMF

This is an example of applying Non-negative Matrix Factorization on a corpus of documents and extract additive models of the topic structure of the corpus. The output is a list of topics, each represented as a list of terms (weights are not shown).

The default parameters (n_samples / n_features / n_topics) should make the example runnable in a couple of tens of seconds. You can try to increase the dimensions of the problem, but be aware that the time complexity is polynomial in NMF. 

References:

http://scikit-learn.org/stable/auto_examples/applications/topics_extraction_with_nmf_lda.html
http://scikit-learn.org/stable/modules/decomposition.html

In [1]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck <L.J.Buitinck@uva.nl>
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.datasets import fetch_20newsgroups

In [2]:
n_samples = 2000
n_features = 1000
n_topics = 10
n_top_words = 20

In [3]:
# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data
print("done in %0.3fs." % (time() - t0))

Loading dataset...


No handlers could be found for logger "sklearn.datasets.twenty_newsgroups"


done in 66.965s.


In [4]:
data_samples[0]


u"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

In [5]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

Extracting tf-idf features for NMF...
done in 3.044s.


In [6]:
tfidf[0]

<1x39115 sparse matrix of type '<type 'numpy.float64'>'
	with 56 stored elements in Compressed Sparse Row format>

In [7]:
print(tfidf[0])

  (0, 33831)	0.0670739217313
  (0, 33307)	0.0944452134759
  (0, 24262)	0.165390978626
  (0, 12059)	0.0633334912554
  (0, 6935)	0.122123425772
  (0, 12220)	0.106451924325
  (0, 33080)	0.09176839573
  (0, 22786)	0.397352856718
  (0, 30569)	0.152250856945
  (0, 19582)	0.323869349055
  (0, 29725)	0.135654431165
  (0, 29228)	0.161934674527
  (0, 27819)	0.098878623467
  (0, 19576)	0.206962724505
  (0, 38004)	0.0731453984357
  (0, 17294)	0.0743825035025
  (0, 21515)	0.106116390108
  (0, 14135)	0.106451924325
  (0, 29036)	0.0996903702203
  (0, 18665)	0.169621152966
  (0, 11803)	0.102632391061
  (0, 21231)	0.101040643532
  (0, 25282)	0.129617443792
  (0, 35470)	0.0717655732032
  (0, 18368)	0.111681219525
  :	:
  (0, 29673)	0.0970408767494
  (0, 9204)	0.0949920933722
  (0, 5926)	0.132066934927
  (0, 31583)	0.132066934927
  (0, 6040)	0.139110735264
  (0, 11106)	0.113802868546
  (0, 29679)	0.0983193738145
  (0, 18960)	0.154248196089
  (0, 4301)	0.106451924325
  (0, 9608)	0.147277168158
  (0, 32431

In [8]:
# Fit the NMF model
print("Fitting the NMF model with tf-idf features,"
      "n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

Fitting the NMF model with tf-idf features,n_samples=2000 and n_features=1000...
done in 1.859s.


In [9]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)


Topics in NMF model:
Topic #0:
don people just like think know good time ve right make say want really did way new use going ll
Topic #1:
windows file dos files window program use running version ms using problem server pc screen ftp run os application software
Topic #2:
god jesus bible christ faith believe christians christian heaven sin hell life church truth lord say belief does existence man
Topic #3:
geb dsl chastity n3jxp cadre shameful pitt intellect skepticism surrender gordon banks soon edu lyme blood weight patients medical probably
Topic #4:
key chip encryption clipper keys escrow government algorithm security secure encrypted public nsa des enforcement law bit privacy secret use
Topic #5:
drive scsi ide drives disk hard controller floppy hd cd mac boot rom cable internal tape bus seagate bios quantum
Topic #6:
game team games players hockey year season play win league teams nhl baseball player detroit toronto runs pitching best playoffs
Topic #7:
thanks mail does know adva

In [10]:
# here are the weights of this document in the 10 topics
# note that they are sparse
nmf.transform(tfidf[0])

array([[ 0.01525578,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.0401674 ,  0.        ]])

## Numpy multiplication refresher

In [11]:
# It is convention to import NumPy with the alias np
import numpy as np

In [12]:
# numpy array with the values 1, 2, 3
data1 = np.array([1, 2, 3])
# Perform the scalar product of 5 and the numpy array
data2 = 5 * data1
print(data1)
print(data2)

[1 2 3]
[ 5 10 15]


In [13]:
# np.dot give the regular matrix multiplication
A = np.arange(6).reshape((3,2))
B = np.arange(4).reshape((2,2))
print(A)
print(B)
print(np.dot(A,B))

[[0 1]
 [2 3]
 [4 5]]
[[0 1]
 [2 3]]
[[ 2  3]
 [ 6 11]
 [10 19]]


In [52]:
# * gives element wise multiplication
a = np.arange(5)
b = np.arange(5)
print(a)
print(b)
print(a*b)

[0 1 2 3 4]
[0 1 2 3 4]
[ 0  1  4  9 16]


## Gradient Descent of MF

In [27]:
# Use the following matrix as an example. Here 0 means "empty". Use k = 2
Y = np.array([[5, 1, 4, 5, 1], [0, 5, 2, 1, 4], [1, 4, 1, 1, 2],\
              [4, 1, 5, 5, 4], [5, 3, 4, 5, 4], [1, 5, 1, 1, 1], \
              [5, 1, 0, 5, 4]])
print(Y)
print(Y.shape)

[[5 1 4 5 1]
 [0 5 2 1 4]
 [1 4 1 1 2]
 [4 1 5 5 4]
 [5 3 4 5 4]
 [1 5 1 1 1]
 [5 1 0 5 4]]
(7, 5)


In [34]:
# initialize the random matrices U and V
n, m = Y.shape
k = 2
U = np.random.uniform(0, 1, size=n*k).reshape(n,k)
V = np.random.uniform(0, 1, size=m*k).reshape(m,k)

In [39]:
R = (Y == 0).astype(int)
print(R)

[[0 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 0 0]]


In [41]:
N = np.sum(Y > 0)
print(N)

33


In [29]:
print(U)
print(V)

[[ 0.90708295  0.68240166]
 [ 0.78080046  0.47960558]
 [ 0.78175046  0.62386719]
 [ 0.85073205  0.2470913 ]
 [ 0.19957364  0.78864122]
 [ 0.84365199  0.90094208]
 [ 0.83929014  0.97663157]]
[[ 0.45212418  0.75575085]
 [ 0.35006305  0.01482054]
 [ 0.90718219  0.76978735]
 [ 0.50597835  0.60867363]
 [ 0.99909563  0.78987642]]


In [33]:
Y_hat = np.dot(U, V.T)

In [ ]:
# compute Mean square error between Y and Y_hat (hint: you have to use R, use matrix operations)


In [ ]:
# Given U and V compute a gradient


In [42]:
# apply gradient descent to this problem for 500 iterations with 
# learning rate 0.01